In [13]:
import numpy as np
from PIL import Image
from ultralytics import YOLO

def predict_single_image(model, image_path, image_size=(224, 224)):
    """
    Predict the class of a single image using a YOLOv8-cls model.
    
    Args:
        model: A loaded YOLOv8-cls model
        image_path: String path to the image file
        image_size: Tuple of (width, height) for input image size, default (224, 224)
    """
    # Open and preprocess the image
    img = Image.open(image_path)
    
    if img.mode != 'RGB':
        img = img.convert('RGB')
    
    # Resize the image - note that PIL takes (width, height)
    img = img.resize(image_size, Image.Resampling.LANCZOS)
    
    # Convert to numpy array and ensure correct dimensions
    img_array = np.array(img)
    
    # Ensure the image has the right shape (height, width, channels)
    # print(f"Image shape before processing: {img_array.shape}")
    
    # Normalize pixel values
    img_array = img_array.astype(np.float32) / 255.0
    
    # Make prediction
    results = model.predict(source=image_path, verbose=False)
    
    # Process results
    # Get the probabilities from the results
    probs = results[0].probs
    predicted_class = int(probs.top1)
    confidence = float(probs.top1conf)
    
    return {
        'predicted_class': predicted_class,
        'confidence': confidence,
        'all_probabilities': probs.data.tolist()
    }

In [ ]:
# Load your model
model = YOLO(r'C:\Users\kanan\Desktop\Project_TMJOA\2D_Pipeline\YOLO\runs\classify\train\weights\best.pt')  # Replace with your model path

# Path to your image
image_path = r'D:\Kananat\TF_TMJOA_jpg_x_5px_test_batch_by_ID\erosion_0\51-3282 R\51-3282 R_x_088.jpg'  # Replace with your image path

# Get prediction
result = predict_single_image(model, image_path)

# Print results in a readable format
print(f"Predicted class: {result['predicted_class']}")
print(f"Confidence: {result['confidence']:.2%}")
print(f"all_probabilities: {result['all_probabilities']}")

In [2]:
import os
from pathlib import Path

In [11]:
def predict_folder_images(model, folder_path):
    """
    Predict classes for all images in a given folder using a YOLOv8-cls model.
    
    Args:
        model: A loaded YOLOv8-cls model
        folder_path: String or Path object pointing to the folder containing images
    
    Returns:
        Dictionary where:
            - keys are image filenames
            - values are prediction dictionaries containing:
                - predicted_class
                - confidence
                - all_probabilities
    """
    # Convert folder_path to Path object for better path handling
    folder_path = Path(folder_path)
    
    # Dictionary to store results for each image
    predictions = {}
    
    # List of common image extensions we want to process
    # We can easily add more extensions if needed
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    
    try:
        # Iterate through all files in the folder
        for file_path in folder_path.iterdir():
            # Check if the file is an image by looking at its extension
            if file_path.suffix.lower() in image_extensions:
                try:
                    # Get predictions for this image
                    result = predict_single_image(model, str(file_path))
                    
                    # Store the result with the filename as key
                    predictions[file_path.name] = result
                    
                    # Print progress (optional, but helpful for monitoring)
                    # print(f"Processed {file_path.name}: Class {result['predicted_class']} "
                          # f"with confidence {result['confidence']:.2%}")
                          
                except Exception as e:
                    # If there's an error with one image, log it but continue processing others
                    print(f"Error processing {file_path.name}: {str(e)}")
                    predictions[file_path.name] = {"error": str(e)}
        
        return predictions
    
    except Exception as e:
        # Handle any errors that might occur when accessing the folder
        raise Exception(f"Error accessing folder {folder_path}: {str(e)}")


In [6]:
# Set up paths
images_folder = r'D:\Kananat\TF_TMJOA_jpg_x_5px_test_batch_by_ID\erosion_1\65-23331 R'  # Replace with your folder path

# Load the model
model = YOLO(r'C:\Users\kanan\Desktop\Project_TMJOA\2D_Pipeline\YOLO\runs\classify\train\weights\best.pt')  # Replace with your model path

# Process all images
results = predict_folder_images(model, images_folder)

# Print a summary of the results
print("\nPrediction Summary:")
print("-" * 50)
for image_name, prediction in results.items():
    if "error" in prediction:
        print(f"{image_name}: Error - {prediction['error']}")
    else:
        print(f"{image_name}: Class {prediction['predicted_class']} "
                f"(Confidence: {prediction['confidence']:.2%})")

Image shape before processing: (224, 224, 3)
Processed 65-23331 R_x_063.jpg: Class 1 with confidence 52.30%
Image shape before processing: (224, 224, 3)
Processed 65-23331 R_x_064.jpg: Class 0 with confidence 52.53%
Image shape before processing: (224, 224, 3)
Processed 65-23331 R_x_065.jpg: Class 0 with confidence 62.12%
Image shape before processing: (224, 224, 3)
Processed 65-23331 R_x_066.jpg: Class 0 with confidence 84.38%
Image shape before processing: (224, 224, 3)
Processed 65-23331 R_x_067.jpg: Class 0 with confidence 94.96%
Image shape before processing: (224, 224, 3)
Processed 65-23331 R_x_068.jpg: Class 1 with confidence 58.82%
Image shape before processing: (224, 224, 3)
Processed 65-23331 R_x_069.jpg: Class 1 with confidence 91.92%
Image shape before processing: (224, 224, 3)
Processed 65-23331 R_x_070.jpg: Class 1 with confidence 98.85%
Image shape before processing: (224, 224, 3)
Processed 65-23331 R_x_071.jpg: Class 1 with confidence 98.70%
Image shape before processin

In [8]:
def make_final_prediction(predictions, confidence_threshold=0.5):
    """
    Make a final binary classification (0 or 1) based on multiple predictions.
    
    This function implements a majority voting system, where the final class
    is determined by which class (0 or 1) appears more frequently in the
    predictions that meet our confidence threshold.
    
    Args:
        predictions: Dictionary where keys are image names and values are prediction
                    dictionaries containing 'predicted_class' and 'confidence'
        confidence_threshold: Minimum confidence level to consider a prediction
                            (default 0.0 means consider all predictions)
    
    Returns:
        Dictionary containing:
        - final_class: The majority class (0 or 1)
        - confidence_score: Proportion of predictions supporting the majority class
        - detailed_stats: Additional statistics about the predictions
    """
    # Initialize counters for each class
    class_counts = {0: 0, 1: 0}
    
    # Count valid predictions (those without errors and above threshold)
    total_valid_predictions = 0
    
    # Keep track of confidence scores for detailed analysis
    confidence_scores = {0: [], 1: []}
    
    # Process each prediction
    for image_name, pred in predictions.items():
        # Skip any predictions that had errors
        if "error" in pred:
            continue
            
        # Get the prediction and confidence
        pred_class = pred['predicted_class']
        confidence = pred['confidence']
        
        # Only count predictions above our confidence threshold
        if confidence >= confidence_threshold:
            class_counts[pred_class] += 1
            confidence_scores[pred_class].append(confidence)
            total_valid_predictions += 1
    
    # If we have no valid predictions, we can't make a decision
    if total_valid_predictions == 0:
        return {
            'final_class': None,
            'confidence_score': 0.0,
            'detailed_stats': {
                'error': 'No valid predictions found',
                'class_counts': class_counts,
                'total_predictions': total_valid_predictions
            }
        }
    
    # Determine which class has the majority
    final_class = 0 if class_counts[0] >= class_counts[1] else 1
    
    # Calculate our confidence in this decision
    # This is the proportion of predictions that agree with our final decision
    confidence_score = class_counts[final_class] / total_valid_predictions
    
    # Calculate average confidence for each class
    avg_confidence = {
        0: sum(confidence_scores[0]) / len(confidence_scores[0]) if confidence_scores[0] else 0,
        1: sum(confidence_scores[1]) / len(confidence_scores[1]) if confidence_scores[1] else 0
    }
    
    # Prepare detailed statistics
    detailed_stats = {
        'class_counts': class_counts,
        'total_predictions': total_valid_predictions,
        'predictions_above_threshold': total_valid_predictions,
        'average_confidence_per_class': avg_confidence,
        'majority_percentage': confidence_score * 100
    }
    
    return {
        'final_class': final_class,
        'confidence_score': confidence_score,
        'detailed_stats': detailed_stats
    }

In [14]:
# Set up paths
images_folder = r'D:\Kananat\TF_TMJOA_jpg_x_5px_test_batch_by_ID\erosion_1\65-23331 R'  # Replace with your folder path

# Load the model
model = YOLO(r'C:\Users\kanan\Desktop\Project_TMJOA\2D_Pipeline\YOLO\runs\classify\train\weights\best.pt')  # Replace with your model path

predictions = predict_folder_images(model, images_folder)

# Make final prediction with a confidence threshold
final_result = make_final_prediction(predictions, confidence_threshold=0.5)

# Print the results in a readable format
print("\nFinal Classification Results:")
print("-" * 50)
print(f"Final Class: {final_result['final_class']}")
print(f"Confidence Score: {final_result['confidence_score']:.2%}")
print("\nDetailed Statistics:")
stats = final_result['detailed_stats']
print(f"Class 0 count: {stats['class_counts'][0]}")
print(f"Class 1 count: {stats['class_counts'][1]}")
print(f"Total valid predictions: {stats['total_predictions']}")
print(f"Average confidence for class 0: {stats['average_confidence_per_class'][0]:.2%}")
print(f"Average confidence for class 1: {stats['average_confidence_per_class'][1]:.2%}")
print(f"Majority class percentage: {stats['majority_percentage']:.1f}%")


Final Classification Results:
--------------------------------------------------
Final Class: 1
Confidence Score: 95.28%

Detailed Statistics:
Class 0 count: 6
Class 1 count: 121
Total valid predictions: 127
Average confidence for class 0: 79.12%
Average confidence for class 1: 97.84%
Majority class percentage: 95.3%


In [15]:
from pathlib import Path
import pandas as pd
from datetime import datetime

def analyze_multiple_folders(base_folder_path, model, confidence_threshold=0.0):
    """
    Analyze all folders within a base folder, making predictions for each folder
    and compiling the results into a comprehensive report.
    
    Args:
        base_folder_path: Path to the folder containing multiple subfolders with images
        model: Loaded YOLOv8-cls model
        confidence_threshold: Minimum confidence level for considering predictions
    
    Returns:
        Dictionary containing results for each folder and a summary DataFrame
    """
    # Convert to Path object for better path handling
    base_path = Path(base_folder_path)
    
    # Dictionary to store results for each folder
    folder_results = {}
    
    # List to store results for DataFrame creation
    results_for_df = []
    
    # Process each subfolder in the base folder
    for folder_path in base_path.iterdir():
        # Skip if it's not a directory
        if not folder_path.is_dir():
            continue
            
        print(f"\nProcessing folder: {folder_path.name}")
        print("-" * 50)
        
        try:
            # Get predictions for all images in this folder
            folder_predictions = predict_folder_images(model, folder_path)
            
            # Make final prediction for this folder
            final_result = make_final_prediction(
                folder_predictions, 
                confidence_threshold=confidence_threshold
            )
            
            # Store detailed results
            folder_results[folder_path.name] = {
                'predictions': folder_predictions,
                'final_result': final_result
            }
            
            # Add to our DataFrame results
            results_for_df.append({
                'Folder': folder_path.name,
                'Final Class': final_result['final_class'],
                'Confidence Score': final_result['confidence_score'],
                'Total Images': final_result['detailed_stats']['total_predictions'],
                'Class 0 Count': final_result['detailed_stats']['class_counts'][0],
                'Class 1 Count': final_result['detailed_stats']['class_counts'][1],
                'Average Conf Class 0': final_result['detailed_stats']['average_confidence_per_class'][0],
                'Average Conf Class 1': final_result['detailed_stats']['average_confidence_per_class'][1]
            })
            
        except Exception as e:
            print(f"Error processing folder {folder_path.name}: {str(e)}")
            folder_results[folder_path.name] = {'error': str(e)}
    
    # Create DataFrame for easy analysis
    results_df = pd.DataFrame(results_for_df)
    
    # Generate summary report
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_path = base_path / f"analysis_report_{timestamp}.csv"
    results_df.to_csv(report_path, index=False)
    
    return {
        'folder_results': folder_results,
        'summary_df': results_df,
        'report_path': report_path
    }

In [17]:
# Set up paths
base_folder = r'D:\Kananat\TF_TMJOA_jpg_x_5px_test_batch_by_ID\erosion_1'  # Replace with your folder path

# Load the model
model = YOLO(r'C:\Users\kanan\Desktop\Project_TMJOA\2D_Pipeline\YOLO\runs\classify\train\weights\best.pt')  # Replace with your model path

# Run analysis
results = analyze_multiple_folders(base_folder, model, confidence_threshold=0.5)

# Print comprehensive results
print("\nAnalysis Complete!")
print("-" * 50)
print(f"Report saved to: {results['report_path']}")
print("\nSummary of Results:")
print(results['summary_df'].to_string())

# Print detailed statistics for each folder
print("\nDetailed Results by Folder:")
print("-" * 50)
for folder_name, folder_data in results['folder_results'].items():
    if 'error' in folder_data:
        print(f"\n{folder_name}: Error - {folder_data['error']}")
        continue
        
    final_result = folder_data['final_result']
    stats = final_result['detailed_stats']
    
    print(f"\nFolder: {folder_name}")
    print(f"Final Class: {final_result['final_class']}")
    print(f"Confidence Score: {final_result['confidence_score']:.2%}")
    print(f"Total Images Processed: {stats['total_predictions']}")
    print(f"Class Distribution: 0: {stats['class_counts'][0]}, "
            f"1: {stats['class_counts'][1]}")


Processing folder: 47-22136 R
--------------------------------------------------

Processing folder: 51-26987 R
--------------------------------------------------

Processing folder: 54-1411 R
--------------------------------------------------

Processing folder: 56-27847 R
--------------------------------------------------

Processing folder: 58-38918 R
--------------------------------------------------

Processing folder: 60-21851 R
--------------------------------------------------

Processing folder: 60-25232 R
--------------------------------------------------

Processing folder: 62-12734 R
--------------------------------------------------

Processing folder: 62-2274 R
--------------------------------------------------

Processing folder: 62-24379 R
--------------------------------------------------

Processing folder: 62-26639 R
--------------------------------------------------

Processing folder: 62-7533 R
--------------------------------------------------

Processing folder: